# CGT57500 / ASM 59100 - Edge Device Lab 4

# Deep Learning-Based Image Classification

### 1. Introduction 
Deep learning (DL) is a subset of machine learning (ML) which is a subset of artificial intellignece (AI). DL uses convolutional neural networks (CNN)s to extract important features from particular datasets that used to train models for learning specific patterns. One of the most popular and widely used applications of deep learning is image classification. Image classification refers to the identification of different objects that are represented within images. DL-based image classification models require large datasets with hundereds of thousands of images for learning features accurately. 
<!-- 
<img src="http://drive.google.com/uc?export=view&id=1Lz9plj4xJY9rxvlFsw-DT4SI0IE-5r3J" width=300/> -->

In order to train image classification models, a dataset comprised of different classes is first provided. Each class consists of hunderds to thousands of images. As the number of images used for training image classification models increases, CNN can help train the models better, resulting in higher accuracies. One of the most popular dataset in the MNIST dataset consisting of 10 classes for the numbers 0 to 9. Each class consists of multiple images. In addition, "Fashion MNIST" is another popular dataset that is commonly used as a benchmast. Both of these datasets will be used in this lab for training deep learning models capable of identifying the classes present in each of these datasets.

The image below shows how a sliding window moves across an image to learn specific features using CNNs.

<!-- <img src="img/cnn1.gif" width=300 height=300 />      -->

### 2. Language and Framework
There are multiple different programming languages and frameworks that are used for training deep learning models. Python is currently the most popular programming language that is used. In addition, two common frameworks are used, namely TensorFlow and PyTorch. However, as TesnorFlow was first introduced, it is more commonly used with a larger community for support. Keras is a framework that is built on top of TensorFlow which helped to easily implement deep learning. Therefore, for the purpose of this lab, Keras and TensorFlow will be used. 

<!-- <img src="img/keras.jpg" width=500 height=200 /> -->

### 3. Import Libraries

In [ ]:
# first import the tensorflow and keras libraries
import tensorflow as tf

In [ ]:
# import additional libraries for arrays and plotting
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import additional libraries
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.utils import img_to_array, array_to_img
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

### 4. Import Dataset
The "MNIST" and "Fashion MNIST" datasets are benchmark datasets that avaialble to download from tensorflow. Run the code block below in order to obtain the dataset in the correct format. The images below  shows how CNN's learn important features from the MNIST dataset.

<!-- <img src="img/cnn2.gif" width=300 height=300 /> -->

In [ ]:
# import the MNIST dataset
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = tf.keras.datasets.mnist.load_data()

In [ ]:
# print the shape of the dataset
x_train_mnist.shape

After displaying the shape of the training dataset, you will see (60000,28,28). Here, 60000 represents the number of images that are present and 28 represents that the width and the height of the images is 28 (28 x 28 pixels).

However, we will not be able to use the dataset with this shape. Therefore, it needs to be resized and normailized. This is because, in the Keras and Tensorflow frameworks, the input needs to be in an array of 4 dimensions while we currently have 3.

In [ ]:
# Resize and Normalize the dataset in order to prepare it for training a deep learning model
x_train_mnist = x_train_mnist.reshape(60000, 28, 28, 1) # the training set is resized and 60000 represents the number of training images
x_test_mnist = x_test_mnist.reshape(10000, 28, 28, 1) # the testing set is resized and 10000 represents the number of training images
input_shape = (28,28,1) # this is the input shape for each of the image that will be used for training

In [ ]:
y_train_mnist.shape

y_train_mnist and y_test_mnist simply represent the labels for each of the image that is present. Therefore, these don't need to be resized.

### 5. Build Model Network Architecture

In [ ]:
# Build the Neural Network Model Architecture

model = tf.keras.Sequential() # use sequential to initialize the model before adding layers

''' Convolutional Layer 1 '''
# we will add our first convolutional layer
model.add(Conv2D(64,  # The first layer has 64 filters
                (3,3),  # This is the kernel size which represents the size of the convolution window. We will use 3 x 3 convolution.
                activation='relu',  # This is the RELU activation function. RELU is commonly used in all the convolutional layers.
                input_shape=input_shape))  # The input shape is only provided to the first layer in the architecture and the following layers will automatically use this. Based on our dataset, this is (28,28,1)


# each convolutional layer is followed by a maxpooling layer
model.add(MaxPooling2D(2,2))  # We will use MaxPooling 2 x 2. This helps combine features in differnet parts of the image.


# add a few more convolutional layers with reduced number of nodes.
''' Convolutional Layer 2 '''
model.add(Conv2D(32,  # The second layer has 32 filters.
                (3,3),  # This is the kernel size which represents the size of the convolution window. We will use 3 x 3 convolution.
                activation='relu'))  # This is the RELU activation function. RELU is commonly used in all the convolutional layers.

# each convolutional layer is followed by a maxpooling layer
model.add(MaxPooling2D(2,2))


''' Convolutional Layer 3 '''
model.add(Conv2D(16,  # The second layer has 16 filters.
                (3,3),  # This is the kernel size which represents the size of the convolution window. We will use 3 x 3 convolution.
                activation='relu'))  # This is the RELU activation function. RELU is commonly used in all the convolutional layers.

# each convolutional layer is followed by a maxpooling layer
model.add(MaxPooling2D(2,2))


''' Flatten '''
# after adding all the convolutional layers, we have to flatten the network in order to make predictions and assign classes
model.add(Flatten())  # flatten will convert the 2D network into a 1D network 

''' Dense Layer 1 '''
# after the architecture is flattened, we have to add fully convolutional layers or dense layers
model.add(Dense(128,  # The dense layer has 128 filters
               activation='relu'))  # This is the RELU activation function

''' Dense Layer 2 / Final Output Layer '''
# finally, we add the output layer where the activation function must be a softmax and the number of nodes must be equal to the number of classes
model.add(Dense(10,  # The final output layer has 10 filters which corresponds to the 10 different classes that we are going to be classifying
               activation='softmax'))  # This is the Softmax activation function which is necessary in the final output layer for multi-class classification problems. Softmax provides a probability for each class.


In [ ]:
# show the model summary
model.summary()

### 6. Set The Hyperparameters and Train The Model

In [ ]:
# Compile the Model
model.compile(optimizer='adam',  # We will be using the Adam optimizer. This is useful and has proven effective for multiclass classification problems.
              loss='sparse_categorical_crossentropy',  # We will use the categorical crossentropy loss function. The loss function helps the different filters update weights to learn features from data.
              metrics=['accuracy'])  # While training the model, this line will hep keep track of the accuracy.

# Train the Model
history = model.fit(x=x_train_mnist, y=y_train_mnist, validation_data=(x_test_mnist, y_test_mnist), epochs=10)
# x represents the training data
# y represnets the training labels
# validation data requires (testing data, testing labels)
# we train the model for 10 epochs which means that the model will go over the entire dataset 10 times to learn and confirm model training performance

### 7. Evaluate The Model and Visualize The Results

In [ ]:
# Evaluate the Model

# print out the testing accuracy
model.evaluate(x_test_mnist, y_test_mnist)

In [ ]:
# shown an example image and pass it into the model
image_index = 4444
plt.imshow(x_test_mnist[image_index].reshape(28, 28),cmap='Greys')
pred = model.predict(x_test_mnist[image_index].reshape(1, 28, 28, 1))
print('The image belongs to the class: ', pred.argmax())

In [ ]:
# plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Save the Model
model.save('mnist_custom.h5')

## Question: What can you say about the training of this model by looking at the accuracy and loss plots?

## Ans:

### 8. Now Repeat The Steps For The Fashion MNIST Dataset

In [ ]:
# import the Fashion MNIST dataset
(x_train_fashion, y_train_fashion), (x_test_fashion, y_test_fashion) = tf.keras.datasets.fashion_mnist.load_data()

The dataset has been imported. Use the same approach as above to resize the dataset, build the model, train the model, and evaluate the model.

In [ ]:
# Resize and Normalize the dataset
# Enter Code Here ...

In [ ]:
# Build the Neural Network Model Architecture
# Enter Code Here ...

In [ ]:
# Train the Model
# Enter Code Here ...

In [ ]:
# Evaluate the Model
# Enter Code Here ...

In [ ]:
# Save the Model
model.save('fashion_mnist_custom.h5')

## Question: What can you say about the training of this model by looking at the accuracy and loss plots?

## Ans:

### 9. Transfer Learning
You must have notices that training your model takes a long time. This is because a model was built and trained from scratch. This means that the model that was created had no prior knowledge and everything was learnt from scratch. Although you obtained high accuracies, this is not always that case due to limited availability of large datasets. Therefore, we introduce the concept of transfer learning. Transfer learning is a technique used for training deep learning models where pre-trained weights are obtained and used as priors for training new models. The pre-trained weights are usually obtained from models that were trained on large datasets with multiple classes. It is not necessary for the pre-trained weights to be obtaine from a model that was trained on the same dataset, nor the same application.

Pre-trained weights can be loaded into your custom defined models from part 5 of this lab. However, over the past decade, multiple netowrk architectures have been developed that can used with the pre-trained weights. For this lab, you will be using the VGG16 network architecture as shown below. This network architecture can be easily loaded from the Keras and TensorFlow libraries. 

<!-- <img src="img/vgg.png" width=500 height=200 /> -->

A common dataset that is used for obtain pre-trained weights is the ImageNet dataset. The pre-trained weights are already avaialble from Keras and TensorFlow. In this section, you will be loading pre-trained weights into popular network architectures for training the same models.

<!-- <img src="img/Imagenet.jpg" width=500 height=200 /> -->

In [ ]:
# load the VGG16 model with the ImageNet pre-trained weights
tf.keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
'''
You may also load additional Network Architectures:
1. ResNet50 architecture: tf.keras.applications.resnet50.ResNet50(...)
2. InceptionV3 architecture: tf.keras.applications.inception_v3.InceptionV3(...)
'''

### Load the dataset in 3 dimensions as per Network Architecture Requriments

In [ ]:
# import the MNIST dataset
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = tf.keras.datasets.mnist.load_data()

In [ ]:
# the code below has been provided to you for preparing the dataset
# this time, the data needs to be in RGB format rather than greyscale therefore there are additional steps for preparing the dataset
x_train_mnist = np.dstack([x_train_mnist] * 3)
x_test_mnist = np.dstack([x_test_mnist] * 3)
x_train_mnist.shape, x_test_mnist.shape

x_train_mnist = x_train_mnist.reshape(-1, 28,28,3)
x_test_mnist = x_test_mnist.reshape (-1,28,28,3)
x_train_mnist.shape, x_test_mnist.shape

x_train_mnist = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_train_mnist])
x_test_mnist = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_test_mnist])
#train_x = preprocess_input(x)

x_train_mnist.shape, x_test_mnist.shape

In [ ]:
# load in the custom VGG16 model and build the architecture
inp = Input((48, 48, 3))
pre_model = tf.keras.applications.VGG16(include_top=False,  # This will exclude the final output layer from the pre-trained VGG16 model as it is comprised of 1000 filters while we need 10 only
                                        weights='imagenet',  # This will import the weights from the ImageNet trained model which will essentially help implement Transfer Learning
                                        input_tensor=inp,  # This is the input tensor size for the images / data
                                        input_shape=(48, 48, 3), # This is the input size for the images / data
                                        pooling='max')  # We will train the model using max pooling 

# show the model summary
pre_model.summary()

In [ ]:
# From the summary above, you will noteice that the final layer at the bottom has size 512. We need an additional layer with 10 filters for the 10 classes in the MNIST dataset.

# Add a final output Dense Layer with 10 filters and Softmax activation function
x = pre_model.output
out = Dense(10, activation='softmax')(x)
complete_model = Model(inp, out)

# show the model summary
complete_model.summary()

In [ ]:
# From the summary above, you will notice that the final layer at the bottom now has size 10

# make sure that the convolutional base is frozen to use the transfer learning weights from ImageNet
for layer in complete_model.layers[:-1]:
    layer.trainable=False

# show the model summary
complete_model.summary()

In [ ]:
# compile the pre-trained model
complete_model.compile(optimizer='adam',  # We will be using the Adam optimizer. This is useful and has proven effective for multiclass classification problems.
              loss='sparse_categorical_crossentropy',  # We will use the categorical crossentropy loss function. The loss function helps the different filters update weights to learn features from data.
              metrics=['accuracy'])  # While training the model, this line will hep keep track of the accuracy.

# train the pre-trained model
history = complete_model.fit(x=x_train_mnist, y=y_train_mnist, batch_size=128, validation_data=(x_test_mnist, y_test_mnist), epochs=10)
# x represents the training data
# y represnets the training labels
# batch size represents the number of images that will be fed into the model at a time (if it increases, training will be faster but it will require more RAM)
# validation data requires (testing data, testing labels)
# we train the model for 10 epochs which means that the model will go over the entire dataset 10 times to learn and confirm model training performance

In [ ]:
# plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Save the Model
complete_model.save('mnist_transfer_learning.h5')

## Question: What can you say about the training of this model by looking at the accuracy and loss plots?

## Ans:

### 10. Now Repeat The Steps For Transfer Learning From Step 9 On The Fashion MNIST Dataset

In [ ]:
# import the Fashion MNIST dataset
(x_train_fashion, y_train_fashion), (x_test_fashion, y_test_fashion) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# Resize and Normalize the dataset
# Enter Code Here ...

In [ ]:
# load the VGG16 model with the ImageNet pre-trained weights
# Enter Code Here ...

In [ ]:
# Train the Model
# Enter Code Here ...

In [ ]:
# Evaluate the Model
# Enter Code Here ...

In [ ]:
# Save the Model
complete_model.save('fashion_mnist_transfer_learning.h5')